In [1]:
%%capture
%pip install dateparser requests datetime
%pip install --upgrade "ibm-watson>=6.0.0"

In [2]:
#import libraries

import requests
import json
import datetime
from datetime import timedelta
import dateparser
import logging
from ibm_watson import DiscoveryV2, ApiException
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
import warnings
import time
from requests.exceptions import Timeout
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
warnings.filterwarnings("ignore")

In [3]:
from icpd_core import icpd_util
SECRET_VAULT_NAME = 'DegreedAPICredentials'
for s in icpd_util.get_my_secrets_v2():
    if s['secret_name'] == SECRET_VAULT_NAME:
        CPD_SECRET_URN = s['secret_urn']

In [4]:
# Define constants and values that should be updated based on env needs
DISCOVERY_URL = 'https://cpd-cp4dpre.apps.cp4d-pre-phx.k8s.boeing.com/discovery/cp4dpre-wd/instances/1651527433601432/api'

DISCOVERY_PROJECT_NAME = "virtual-assistant-poc"
PATHWAY_COLLECTION_NAME = "pathways"
VIDEO_COLLECTION_NAME = "videos"

In [5]:
# Below variables do not need to be changed frequently.

DISCOVERY_VERSION = '2020-08-30'

TOKEN_URL = "https://degreed.com/oauth/token"
TOKEN_PAYLOAD = "grant_type=client_credentials" \
            "&client_id={}" \
            "&client_secret={}" \
            "&scope=users%3Aread%2Cusers%3Awrite%2Ccontent%3Aread%2Ccontent%3Awrite%2Ccompletions%3Aread%2Ccompletions%3Awrite%2Crequired_learning%3Awrite%2Crequired_learning%3Aread%2Cpathways%3Aread"
PATHWAYS_URL = "https://api.degreed.com/api/v2/pathways?limit=1000&next="
VIDEO_URL = "https://api.degreed.com/api/v2/content/videos?limit=1000&filter[internal_only]=true"


In [6]:
def get_secret_from_cpd_internal_vault():
    """to get secrets from the vault"""
    degreed_api_credentials=icpd_util.get_my_secret_details_v2(CPD_SECRET_URN)['data']['secret']['generic']['value']['secretGenericAuth']
    return degreed_api_credentials

In [7]:
try:
        secret = get_secret_from_cpd_internal_vault()
        logging.info("Successfully retrieved secrets")
except Exception as e:
        logging.error(f'Error while retrieving secret information...')

CLIENT_ID = secret['clientId']
CLIENT_SECRET = secret['clientSecret']
DISCOVERY_BEARER_TOKEN = secret['bearerToken']

Response.json <Response [200]>
INFO:root:Successfully retrieved secrets


In [8]:
# All functions to call to degreed API

def get_access_token(TOKEN_URL, TOKEN_PAYLOAD, CLIENT_ID, CLIENT_SECRET):
    """ returns the api token required to make requests to degreed"""
    
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    logging.info("Getting access token...")

    retries = 1
    while retries < 4:
        try:
            return requests.post(TOKEN_URL, data=TOKEN_PAYLOAD.format(CLIENT_ID,CLIENT_SECRET), headers=headers, timeout=10).json()
        except requests.exceptions.Timeout:
            wait = retries * 10
            logging.warning(f'Error: Waiting {wait} seconds and retrying....')
            time.sleep(wait)
            retries += 1
        except Exception as e:
            
            logging.critical('Unable to retrieve Access Token')
            logging.critical(e)
    logging.critical('Unable to retrieve Access Token')

def get_pathways(token, PATHWAYS_URL):

    pathways_headers = {'Accept': 'application/json',
        'Authorization': token['token_type'].title() + ' ' + token['access_token'] }

    payload={}

    retries = 1
    success = False
    while not success and retries < 4:
        try:
            response = requests.request("GET", PATHWAYS_URL, headers=pathways_headers, data=payload, timeout=10).json()
            all_pathway = response['data']
            success = True
        except requests.exceptions.Timeout:
            wait = retries * 10
            logging.warning(f'Error: Waiting {wait} seconds and retrying....')
            time.sleep(wait)
            retries += 1
        except requests.exceptions.TooManyRedirects:
            logging.critical("URL is invalid, try a different one")
        except:
             logging.critical("Error occurred while retrieving pathways")
    if success == False:
         logging.critical('Unable to get pathways')
   
    
    try:
        next_url = response["links"]["next"]
        while next_url:
            response = requests.request("GET", next_url, headers=pathways_headers, data=payload, timeout=10).json()
            all_pathway= all_pathway+response['data']

            next_url = response["links"]["next"]
    except KeyError:
        logging.info("Found end of pathway trail.")
        success = True
    except Exception as e:
        logging.critical("Error occurred while retrieving pathways")
        logging.critical(e)
            
    logging.info(f'Total number of pathways is: {len(all_pathway)}')
    return all_pathway

def process_pathways(data):    
    all_pathway = []
    modified_pathway = []
    processed_pathway ={}
    yesterday_datetime = datetime.datetime.now() - timedelta(days=1)
    for i in data:
        data_point = {}
        data_point["id"] = i["id"]
        data_point["title"] = i["attributes"]["title"]
        data_point["text"] = i["attributes"]["summary"]
        data_point["degreed-url"] = i["attributes"]["degreed-url"]
        data_point["modified-at"] = i["attributes"]["modified-at"]
        data_point["type"] = i["type"]

        all_pathway.append(data_point)
        if dateparser.parse(i["attributes"]["modified-at"]) >= yesterday_datetime:
            modified_pathway.append(data_point)
        
    processed_pathway['all_pathway'] = all_pathway
    processed_pathway['modified_pathway'] = modified_pathway
    logging.info(f'Number of pathways found in this pass: {len(all_pathway)} and {len(modified_pathway)} are updated on {yesterday_datetime}')
    return processed_pathway

def write_to_output_file(input, output_filename):
    try:
        output_file = open(output_filename, "w")

        file_input = json.dumps(input, indent = 2)

        output_file.write(file_input)
    except Exception:
        logging.error(f'Error while writing to output file:{ output_filename}')
        
def get_videos(token, VIDEO_URL):

    video_headers = {'Accept': 'application/json',
        'Authorization': token['token_type'].title() + ' ' + token['access_token'] }

    payload={}

    retries = 1
    success = False
    while not success and retries < 4:
        try:
            response = requests.request("GET", VIDEO_URL, headers=video_headers, data=payload, timeout=10).json()
            all_video = response['data']
            success = True
        except requests.exceptions.Timeout:
            wait = retries * 10
            logging.warning(f'Error: Waiting {wait} seconds and retrying....')
            time.sleep(wait)
            retries += 1
        except requests.exceptions.TooManyRedirects:
            logging.critical("URL is invalid, try a different one")
        except:
             logging.critical("Error occurred while retrieving videos")
    if success == False:
         logging.critical('Unable to get videos')
   
    
    try:
        next_url = response["links"]["next"]
        while next_url:
            response = requests.request("GET", next_url, headers=video_headers, data=payload, timeout=10).json()
            all_video= all_video+response['data']

            next_url = response["links"]["next"]
    except KeyError:
        logging.info("Found end of video trail.")
        success = True
    except Exception as e:
        logging.critical("Error occurred while retrieving videos")
        logging.critical(e)
            
    logging.info(f'Total number of videos is: {len(all_video)}')
    return all_video

def process_videos(data):    
    all_video = []
    modified_video = []
    processed_video ={}
    yesterday_datetime = datetime.datetime.now() - timedelta(days=1)
    for i in data:
        data_point = {}
        data_point["id"] = i["id"]
        data_point["title"] = i["attributes"]["title"]
        data_point["text"] = i["attributes"]["summary"]
        data_point["degreed-url"] = i["attributes"]["degreed-url"]
        data_point["image-url"] = i["attributes"]["image-url"]
        data_point["modified-at"] = i["attributes"]["modified-at"]
        data_point["type"] = i["type"]

        all_video.append(data_point)
        if dateparser.parse(i["attributes"]["modified-at"]) >= yesterday_datetime:
            modified_video.append(data_point)
        
    processed_video['all_video'] = all_video
    processed_video['modified_video'] = modified_video
    logging.info(f'Number of videos found in this pass: {len(all_video)} and {len(modified_video)} are updated on {yesterday_datetime}')
    return processed_video

In [9]:
# All functions to call for discovery services

def get_discovery_instance(DISCOVERY_URL, DISCOVERY_VERSION, DISCOVERY_BEARER_TOKEN):
    """to get Discovery Instance"""
    logging.info("Getting Discovery Instance...")

    try:

        authenticator = BearerTokenAuthenticator(DISCOVERY_BEARER_TOKEN)

        discovery = DiscoveryV2(
            version=DISCOVERY_VERSION,
            authenticator=authenticator
        )
    except Exception:
        logging.critical("Exception occured while initializing discovery instance")

    discovery.set_service_url(DISCOVERY_URL)

    discovery.set_disable_ssl_verification(True)

    return discovery
        
def get_project_id(project_name):
    discovery = get_discovery_instance(DISCOVERY_URL, DISCOVERY_VERSION, DISCOVERY_BEARER_TOKEN)

    logging.info("Getting Project ID...")
    
    try:
        projects = discovery.list_projects().result["projects"]

        for project in projects:
            if project["name"] == project_name:
                logging.info(f'Successfully retrieved project id {project["project_id"]}')
                return project["project_id"]
    except ApiException as e:
        logging.exception("Method failed with status code " + str(e.code) + ": " + e.message)

def get_collection_id(COLLECTION_NAME, project_id):
    discovery = get_discovery_instance(DISCOVERY_URL, DISCOVERY_VERSION, DISCOVERY_BEARER_TOKEN)
    try:
        collections = discovery.list_collections(project_id=project_id).get_result()['collections']

        if collections:
            for collection in collections:
                if collection["name"] == COLLECTION_NAME:
                    collection_id = collection["collection_id"]
                    logging.info(f'Successfully retrieved collection id {collection["collection_id"]}')
                    return collection_id
        
        collection_id = discovery.create_collection(
            project_id=project_id,
            name=COLLECTION_NAME
        ).get_result()['collection_id']
        return collection_id

    except ApiException as e:
        logging.critical("Retrieving collection id failed with status code " + str(e.code) + ": " + e.message)

def add_document(discovery, data, file_name, project_id, collection_id):
    
    try:
        #with open(file_name, 'rb') as fileinfo:
        response = discovery.add_document(
            project_id=project_id,
            collection_id=collection_id,
            file=json.dumps(data, indent=2).encode('utf-8'),
            filename=file_name,
            file_content_type='application/json'
        ).get_result()
    except ApiException as e:
        logging.critical("Method failed with status code " + str(e.code) + ": " + e.message)

def update_documents(discovery, data, project_id, collection_id):

    try:
        get_doc_result = discovery.query(project_id=project_id,
                                         collection_ids = [collection_id],
                                         query='id::{}'.format(data["id"])).get_result()            
    except ApiException as e:
        logging.exception("Method failed not retrieving documents for this id with status code " + str(e.code) + ": " + e.message)

    try:
        if get_doc_result['matching_results'] == 1:
            document_id = get_doc_result['results'][0]['document_id']
            response = discovery.update_document(
                project_id=project_id,
                collection_id=collection_id,
                document_id=document_id,
                file=json.dumps(data, indent=2).encode('utf-8'),
                filename=data["id"],
                file_content_type='application/json'
            ).get_result()
        elif get_doc_result['matching_results'] == 0:
            add_document(discovery, data, data['id'], project_id, collection_id)
        else: # > 1  multiple matches
            logging.critical('There are multiple matches, cannot update document')

    except ApiException as e:
        logging.error("Method failed with status code " + str(e.code) + ": " + e.message)
    

In [10]:
import os
os.environ['http_proxy'] = "http://www-proxy-us.boeing.com:31060"
os.environ['https_proxy'] = "http://www-proxy-us.boeing.com:31060"

In [11]:
### check if token is expired ( how to check if token is expired? ), then grab new token and place in yaml file o/w take from yaml file
token = get_access_token(TOKEN_URL, TOKEN_PAYLOAD, CLIENT_ID, CLIENT_SECRET)
logging.info("Access Token Retrieved Successfully")

pathways_headers = {'Accept': 'application/json',
        'Authorization': token['token_type'].title() + ' ' + token['access_token'] }

payload={}

project_id = get_project_id(DISCOVERY_PROJECT_NAME)
pathway_collection_id = get_collection_id(PATHWAY_COLLECTION_NAME, project_id)
video_collection_id = get_collection_id(VIDEO_COLLECTION_NAME, project_id)

INFO:root:Getting access token...
INFO:root:Access Token Retrieved Successfully
INFO:root:Getting Discovery Instance...
INFO:root:Getting Project ID...
INFO:root:Successfully retrieved project id a5556480-4fec-4dff-8926-64a3326fa130
INFO:root:Getting Discovery Instance...
INFO:root:Successfully retrieved collection id ed0efdb5-2c97-10bd-0000-018188d091da
INFO:root:Getting Discovery Instance...
INFO:root:Successfully retrieved collection id ed0efdb5-2c97-10bd-0000-018188d098d4


## Pathway processing

In [12]:
###### Get all pathways and Upload to discovery
today = str(datetime.datetime.now().date())

raw_pathway = get_pathways(token, PATHWAYS_URL)

processed_pathway = process_pathways(raw_pathway)
logging.info(f'Total number of pathways is {len(processed_pathway["all_pathway"])}  and  {len(processed_pathway["modified_pathway"])} were modified')

discovery = get_discovery_instance(DISCOVERY_URL, DISCOVERY_VERSION, DISCOVERY_BEARER_TOKEN)
document_count = discovery.query(project_id = project_id, collection_ids=[pathway_collection_id]).get_result()['matching_results']

if document_count == 0:
    """For the project_id and collection_id, there is no document in the collection...so ADD DOCUMENT"""
    logging.info(f'There are no documents in this collection: {pathway_collection_id}, adding documents...')
    add_document(discovery, processed_pathway['all_pathway'], "all_pathways.json", project_id, pathway_collection_id)
    logging.info(f'Successfully added {len(processed_pathway["all_pathway"])} documents')
    
elif document_count > 0:
    """For the project_id and collection_id, there are already documents in the collection...so UPDATE documents"""
    logging.info(f'There are {document_count} documents in this collection: {pathway_collection_id} already, updating {len(processed_pathway["modified_pathway"])} documents...')
    for i in processed_pathway['modified_pathway']:
        update_documents(discovery, i, project_id, pathway_collection_id)
    logging.info(f'Successfully updated documents {len(processed_pathway["modified_pathway"])}')
    
else:
    logging.critical(f'Something is wrong with this collection {pathway_collection_id}')


#write_to_output_file(processed_pathway['all_pathway'], "all_pathway_{}.json".format(today))
#write_to_output_file(processed_pathway['modified_pathway'], "modified_pathway_{}.json".format(today))


INFO:root:Found end of pathway trail.
INFO:root:Total number of pathways is: 2357
INFO:root:Number of pathways found in this pass: 2357 and 11 are updated on 2022-06-21 00:14:35.771372
INFO:root:Total number of pathways is 2357  and  11 were modified
INFO:root:Getting Discovery Instance...
INFO:root:There are 2357 documents in this collection: ed0efdb5-2c97-10bd-0000-018188b65fc6 already, updating 11 documents...
INFO:root:Successfully updated documents 11


## Video processing

In [13]:
raw_video = get_videos(token, VIDEO_URL)

processed_video = process_videos(raw_video)
logging.info(f'Total number of videos is {len(processed_video["all_video"])}  and  {len(processed_video["modified_video"])} were modified')

discovery = get_discovery_instance(DISCOVERY_URL, DISCOVERY_VERSION, DISCOVERY_BEARER_TOKEN)
document_count = discovery.query(project_id = project_id, collection_ids=[video_collection_id]).get_result()['matching_results']

n = 100000
final_processed_video = [processed_video['all_video'][i * n:(i + 1) * n] for i in range((len(processed_video['all_video']) + n - 1) // n )] 

if document_count == 0:
    """For the project_id and collection_id, there is no document in the collection...so ADD DOCUMENT"""
    logging.info(f'There are no documents in this collection: {video_collection_id}, adding documents...')
    for i in final_processed_video:
        add_document(discovery, i, "all_video.json", project_id, video_collection_id)
    logging.info(f'Successfully added {len(processed_video["all_video"])} documents in {len(final_processed_video)} uploads')
    
elif document_count > 0:
    """For the project_id and collection_id, there are already documents in the collection...so UPDATE documents"""
    logging.info(f'There are {document_count} documents in this collection: {video_collection_id} already, updating {len(processed_video["modified_video"])} documents...')
    for i in processed_video['modified_video']:
        update_documents(discovery, i, project_id, video_collection_id)
    logging.info(f'Successfully updated documents {len(processed_video["modified_video"])}')
    
else:
    logging.critical(f'Something is wrong with this collection {video_collection_id}')

INFO:root:Found end of video trail.
INFO:root:Total number of videos is: 146398
INFO:root:Number of videos found in this pass: 146398 and 24 are updated on 2022-06-21 00:19:17.347485
INFO:root:Total number of videos is 146398  and  24 were modified
INFO:root:Getting Discovery Instance...
INFO:root:There are 42499 documents in this collection: ed0efdb5-2c97-10bd-0000-018188b66650 already, updating 24 documents...
INFO:root:Successfully updated documents 24
